## Response Ranking

 

# Preliminaries: Inspect and Set up environment

In [2]:
import datetime
print(datetime.datetime.now())

2025-07-26 23:02:27.155765


In [62]:
import sys
print(sys.executable)


c:\Users\Owner\Documents\GitProjects\Campaign Response Ranking\venv_main\Scripts\python.exe


In [5]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 0. Data Loading and Inspection


# 0.1 Load data

In [ ]:
import zipfile
import py7zr
import pandas as pd
from pathlib import Path

import zipfile
from pathlib import Path

import zipfile
from pathlib import Path
import shutil
zip_path = r"C:\Users\Owner\Documents\GitProjects\Campaign Response Ranking\kkbox-churn-prediction-challenge.zip"


import zipfile
import py7zr
import pandas as pd
from pathlib import Path

def load_csvs_from_zip_of_7z(zip_path_str, temp_dir_str="unzipped_temp"):
    zip_path = Path(zip_path_str)
    temp_dir = Path(temp_dir_str)
    temp_dir.mkdir(parents=True, exist_ok=True)

    print(f"\n📦 Extracting .7z files from ZIP: {zip_path.name}")
    with zipfile.ZipFile(zip_path, 'r') as z:
        for name in z.namelist():
            if name.endswith('.7z'):
                print(f"→ Extracting {name}")
                z.extract(name, path=temp_dir)

    dfs = {}

    print("\n📂 Extracting .csv files from each .7z:")
    for seven_z_path in temp_dir.rglob("*.7z"):
        print(f"→ Unpacking: {seven_z_path.name}")
        inner_extract_dir = seven_z_path.with_suffix("")  # folder with same name
        inner_extract_dir.mkdir(exist_ok=True)

        try:
            with py7zr.SevenZipFile(seven_z_path, mode='r') as archive:
                archive.extractall(path=inner_extract_dir)
        except Exception as e:
            print(f"❌ Failed to extract {seven_z_path.name}: {e}")
            continue

        for csv_file in inner_extract_dir.glob("*.csv"):
            try:
                df = pd.read_csv(csv_file)
                key = csv_file.stem.lower()  # use file name without extension as key
                dfs[key] = df
                print(f"✓ Loaded {csv_file.name}, shape: {df.shape}")
            except Exception as e:
                print(f"⚠️ Failed to read {csv_file.name}: {e}")

    if not dfs:
        print("⚠️ No CSVs loaded. Check if the ZIP contains .7z files and they contain .csv files.")
    else:
        print(f"\n✅ Loaded {len(dfs)} CSV file(s): {list(dfs.keys())}")

    return dfs





with zipfile.ZipFile(zip_path, 'r') as z:
    print("📦 Files in ZIP:")
    for name in z.namelist():
        print(" -", name)


📦 Files in ZIP:
 - WSDMChurnLabeller.scala
 - members_v3.csv.7z
 - sample_submission_v2.csv.7z
 - sample_submission_zero.csv.7z
 - train.csv.7z
 - train_v2.csv.7z
 - transactions.csv.7z
 - transactions_v2.csv.7z
 - user_logs.csv.7z
 - user_logs_v2.csv.7z


In [55]:
dfs = load_csvs_from_zip_of_7z(
    zip_path_str=r"C:\Users\Owner\Documents\GitProjects\Campaign Response Ranking\kkbox-churn-prediction-challenge.zip",
    temp_dir_str=r"C:\Users\Owner\Documents\GitProjects\Campaign Response Ranking\data"
)


📦 Extracting .7z files from ZIP: kkbox-churn-prediction-challenge.zip
→ Extracting members_v3.csv.7z
→ Extracting sample_submission_v2.csv.7z
→ Extracting sample_submission_zero.csv.7z
→ Extracting train.csv.7z
→ Extracting train_v2.csv.7z
→ Extracting transactions.csv.7z
→ Extracting transactions_v2.csv.7z
→ Extracting user_logs.csv.7z
→ Extracting user_logs_v2.csv.7z

📂 Extracting .csv files from each .7z:
→ Unpacking: members_v3.csv.7z
✓ Loaded members_v3.csv, shape: (6769473, 6)
→ Unpacking: sample_submission_v2.csv.7z
→ Unpacking: sample_submission_zero.csv.7z
✓ Loaded sample_submission_zero.csv, shape: (970960, 2)
→ Unpacking: train.csv.7z
✓ Loaded train.csv, shape: (992931, 2)
→ Unpacking: train_v2.csv.7z
→ Unpacking: transactions.csv.7z
✓ Loaded transactions.csv, shape: (21547746, 9)
→ Unpacking: transactions_v2.csv.7z
→ Unpacking: user_logs.csv.7z
⚠️ Failed to read user_logs.csv: Unable to allocate 2.92 GiB for an array with shape (392106543,) and data type uint64
→ Unpacking:

In [56]:
print(type(dfs))
print(dfs.keys()) 
print(type(dfs['train'])) 

<class 'dict'>
dict_keys(['members_v3', 'sample_submission_zero', 'train', 'transactions'])
<class 'pandas.core.frame.DataFrame'>


In [63]:
# 指定保存路径
output_dir = Path(r"C:\Users\Owner\Documents\GitProjects\Campaign Response Ranking\data")
output_dir.mkdir(parents=True, exist_ok=True)

# 保存 train.csv
dfs['train'].to_csv(output_dir / "train.csv", index=False)

# 保存 members_v3.csv
dfs['members_v3'].to_csv(output_dir / "members_v3.csv", index=False)
# 保存 members_v3.csv
dfs['sample_submission_zero'].to_csv(output_dir / "sample_submission_zero.csv", index=False)
# 保存 members_v3.csv
dfs['transactions'].to_csv(output_dir / "transactions.csv", index=False)

print("✅ Files saved to:", output_dir)


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Owner\\Documents\\GitProjects\\Campaign Response Ranking\\data\\train.csv'